# Compute the value of LEGO Bricks a store has in stock 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re

qua = re.compile('Quantity: (\d+\,?\d*)')
pri = re.compile('Price: EUR (\d+\.\d+)')

url_1 = 'https://store.bricklink.com/HeldderSteine?p=HeldderSteine#/shop?o={%22sort%22:6,%22desc%22:1,%22pgSize%22:100,%22pg%22:3,%22pg%22:'
url_2 = ',%22itemType%22:%22P%22,%22showHomeItems%22:0}'

In [2]:
driver = webdriver.Firefox()

data = []
for i in tqdm(range(1, 25)):
    driver.get(url_1+ str(i) +url_2)
    bs = BeautifulSoup(driver.page_source, 'lxml')
    
    for elem in bs.findAll('article'):
        part = elem.find('ul', {'class', 'bl-breadcrumb'}).findAll('a')[-1].get_text()
        text = elem.find('div', {'class':'buy'}).get_text()
        quantity = re.search(qua, text)[1]
        price = re.search(pri, text)[1]
        data.append([part, quantity, price])

driver.close()

100%|██████████| 24/24 [00:38<00:00,  1.17it/s]


In [3]:
df = pd.DataFrame(data, columns=['Part', 'Quantity', 'Price'])
df['Quantity'] = df['Quantity'].apply(lambda x: x.replace(',', '')).astype(int)
df['Price'] = df['Price'].astype(float)
df.head(2)

,Part,Quantity,Price
0,42056stk01,1,15.0
1,42030stk01,6,10.0


In [4]:
print('Lagerbestand:\t{:.2f}€'.format((df['Quantity']*df['Price']).sum()))

Lagerbestand:	20553.50€
